# Learning 1: Resolution Fundamentals - The Abbe Diffraction Limit

**Duration**: 20-30 minutes  
**Level**: Beginner  
**Prerequisites**: Basic optics knowledge (wavelength, lenses)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Calculate** the Abbe diffraction limit given wavelength (λ) and numerical aperture (NA)
2. **Explain** how NA and wavelength affect resolution
3. **Identify** differences between air, water, and oil immersion objectives
4. **Understand** the Nyquist sampling criterion for digital imaging
5. **Predict** which objective to use for a given sample

---

## Why Resolution Matters

Resolution is the ability to distinguish two closely spaced objects as separate entities. In microscopy, this is fundamentally limited by **diffraction** - the wave nature of light.

No matter how perfect your optics, there's a fundamental limit to how small you can see. Understanding this limit helps you:
- Choose the right objective for your sample
- Set realistic expectations for your experiments
- Understand when super-resolution techniques are needed

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# SPIDS imports
from prism.config.constants import um
from prism.core import create_usaf_target
from prism.core.instruments import create_instrument
from prism.scenarios import get_scenario_preset, list_scenario_presets


# For interactive widgets (optional - graceful fallback if not available)
try:
    import ipywidgets as widgets
    from IPython.display import display

    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("Note: ipywidgets not available. Interactive elements will use static displays.")

# Plotting style
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 11

print("Setup complete!")

---

# Section 1: The Abbe Diffraction Limit

## The Physics of Resolution

When light passes through a lens aperture, it **diffracts** - spreads out due to wave interference. This means a point source doesn't appear as a perfect point, but as a blurry spot called the **Airy disk**.

The **Abbe diffraction limit** describes the minimum distance between two points that can still be resolved:

$$\Delta x = \frac{0.61 \lambda}{\text{NA}}$$

Where:
- $\Delta x$ = minimum resolvable distance (resolution)
- $\lambda$ = wavelength of light
- NA = Numerical Aperture of the objective

### What is Numerical Aperture (NA)?

$$\text{NA} = n \cdot \sin(\theta)$$

Where:
- $n$ = refractive index of the medium (air=1.0, water=1.33, oil=1.515)
- $\theta$ = half-angle of the light cone entering the objective

**Higher NA = Better Resolution** (can collect more light at steeper angles)

## Interactive Resolution Calculator

Let's explore how wavelength and NA affect resolution:

In [ ]:
def calculate_resolution(wavelength_nm: float, na: float) -> dict:
    """
    Calculate Abbe diffraction limit and related parameters.

    Parameters
    ----------
    wavelength_nm : float
        Wavelength in nanometers
    na : float
        Numerical aperture

    Returns
    -------
    dict
        Dictionary with resolution metrics
    """
    # Abbe lateral resolution
    resolution_nm = 0.61 * wavelength_nm / na

    # Spatial frequency cutoff
    freq_cutoff = na / (wavelength_nm * 1e-9)  # cycles/meter
    freq_cutoff_lpmm = freq_cutoff / 1000  # line pairs per mm

    # Interpret the resolution
    if resolution_nm < 250:
        interpretation = "Excellent - Can resolve subcellular organelles, viruses"
    elif resolution_nm < 400:
        interpretation = "Good - Can resolve bacteria, cellular structures"
    elif resolution_nm < 700:
        interpretation = "Moderate - Can resolve large cells, tissue structures"
    else:
        interpretation = "Low - Suitable for tissue overview, large features"

    return {
        "resolution_nm": resolution_nm,
        "resolution_um": resolution_nm / 1000,
        "freq_cutoff_lpmm": freq_cutoff_lpmm,
        "interpretation": interpretation,
    }


# Display function for calculator
def display_resolution(wavelength_nm: float, na: float):
    """Display resolution calculation results."""
    result = calculate_resolution(wavelength_nm, na)

    print(f"\n{'=' * 60}")
    print("  ABBE DIFFRACTION LIMIT CALCULATION")
    print(f"{'=' * 60}")
    print("  Input:")
    print(f"    Wavelength (λ): {wavelength_nm:.0f} nm")
    print(f"    Numerical Aperture (NA): {na:.2f}")
    print("")
    print("  Resolution Limit:")
    print(f"    Δx = 0.61 × {wavelength_nm:.0f} / {na:.2f}")
    print(f"    Δx = {result['resolution_nm']:.1f} nm = {result['resolution_um']:.3f} µm")
    print("")
    print(f"  Spatial Frequency Cutoff: {result['freq_cutoff_lpmm']:.0f} lp/mm")
    print("")
    print("  Interpretation:")
    print(f"    → {result['interpretation']}")
    print(f"{'=' * 60}\n")


# Interactive widget version
if WIDGETS_AVAILABLE:
    wavelength_slider = widgets.FloatSlider(
        value=550,
        min=400,
        max=700,
        step=10,
        description="λ (nm):",
        continuous_update=False,
        style={"description_width": "80px"},
    )
    na_slider = widgets.FloatSlider(
        value=0.9,
        min=0.1,
        max=1.5,
        step=0.05,
        description="NA:",
        continuous_update=False,
        style={"description_width": "80px"},
    )

    output = widgets.interactive_output(
        display_resolution, {"wavelength_nm": wavelength_slider, "na": na_slider}
    )

    display(
        widgets.VBox(
            [widgets.HTML("<h4>Adjust parameters:</h4>"), wavelength_slider, na_slider, output]
        )
    )
else:
    # Static display for environments without widgets
    print("Example calculations:")
    display_resolution(550, 0.9)  # Default values

### Explore Different Scenarios

Try these combinations to build intuition:

In [ ]:
# Common wavelengths and their colors
wavelengths = {
    "Blue (450nm)": 450,
    "Green (550nm)": 550,
    "Red (650nm)": 650,
    "Near-IR (800nm)": 800,
}

# Common objective NAs
objectives = {"10x/0.3 air": 0.3, "40x/0.9 air": 0.9, "60x/1.2 water": 1.2, "100x/1.4 oil": 1.4}

# Create comparison table
print("Resolution (nm) for different wavelength/NA combinations:")
print("=" * 75)
print(f"{'Objective':<20}", end="")
for name in wavelengths:
    print(f"{name:^14}", end="")
print()
print("-" * 75)

for obj_name, na in objectives.items():
    print(f"{obj_name:<20}", end="")
    for wl_name, wl in wavelengths.items():
        res = calculate_resolution(wl, na)["resolution_nm"]
        print(f"{res:^14.0f}", end="")
    print()

print("=" * 75)
print("\nKey insights:")
print("  • Shorter wavelengths → better resolution (blue > red)")
print("  • Higher NA → better resolution (oil > water > air)")
print("  • NA has a larger effect than wavelength in practice")

---

# Section 2: Point Spread Functions (PSF)

## What is a PSF?

The **Point Spread Function (PSF)** describes how the optical system images a single point source. Due to diffraction, a point becomes a blurry spot - the **Airy pattern**.

The PSF is the **fingerprint** of your optical system:
- Its width determines resolution
- Its shape affects image quality
- It's used in deconvolution algorithms

### PSF Width and Resolution

The **Full Width at Half Maximum (FWHM)** of the PSF is related to, but smaller than, the Abbe limit:
- **FWHM** measures where intensity drops to 50% of peak
- **Abbe limit** (0.61λ/NA) is where two point sources can just be distinguished

The Abbe criterion accounts for the overlap of two neighboring Airy patterns, making it slightly larger than the FWHM.

In [ ]:
def generate_psf_for_na(
    na: float, n_medium: float = 1.0, wavelength: float = 550e-9, n_pixels: int = 512
) -> tuple:
    """
    Generate PSF for given NA and medium.

    Parameters
    ----------
    na : float
        Numerical aperture
    n_medium : float
        Refractive index of medium
    wavelength : float
        Wavelength in meters
    n_pixels : int
        Grid size

    Returns
    -------
    tuple
        (psf_2d, psf_1d, x_coords_nm, fwhm_nm)
    """
    # Create frequency grid
    # Pixel size chosen to properly sample the PSF
    abbe_limit = 0.61 * wavelength / na
    pixel_size = abbe_limit / 4  # Oversample PSF

    # Frequency coordinates
    freq = np.fft.fftfreq(n_pixels, pixel_size)
    fx, fy = np.meshgrid(freq, freq)
    f_r = np.sqrt(fx**2 + fy**2)

    # Pupil function (circular aperture)
    cutoff_freq = na / wavelength
    pupil = (f_r <= cutoff_freq).astype(float)

    # PSF is |FT(pupil)|^2
    psf_field = np.fft.ifft2(np.fft.ifftshift(pupil))
    psf_2d = np.abs(psf_field) ** 2
    psf_2d = np.fft.fftshift(psf_2d)  # Center the PSF
    psf_2d = psf_2d / psf_2d.max()  # Normalize

    # Extract 1D profile through center
    center = n_pixels // 2
    psf_1d = psf_2d[center, :]

    # Spatial coordinates in nm
    x_coords = (np.arange(n_pixels) - center) * pixel_size * 1e9

    # Calculate FWHM
    half_max = 0.5
    above_half = psf_1d >= half_max
    if above_half.any():
        indices = np.where(above_half)[0]
        fwhm_pixels = indices[-1] - indices[0]
        fwhm_nm = fwhm_pixels * pixel_size * 1e9
    else:
        fwhm_nm = 0

    return psf_2d, psf_1d, x_coords, fwhm_nm


print("PSF generation function ready.")

## PSF Gallery: Comparing Different Objectives

Let's visualize PSFs for three common objective types:

In [ ]:
# Define objectives to compare
objectives_to_compare = [
    {"name": "10x/0.3 Air", "na": 0.3, "n": 1.0, "color": "blue"},
    {"name": "40x/0.9 Air", "na": 0.9, "n": 1.0, "color": "green"},
    {"name": "100x/1.4 Oil", "na": 1.4, "n": 1.515, "color": "red"},
]

wavelength = 550e-9  # Green light

# Generate PSFs
psf_data = []
for obj in objectives_to_compare:
    psf_2d, psf_1d, x_coords, fwhm = generate_psf_for_na(obj["na"], obj["n"], wavelength)
    abbe_limit = 0.61 * wavelength * 1e9 / obj["na"]  # in nm
    psf_data.append(
        {
            **obj,
            "psf_2d": psf_2d,
            "psf_1d": psf_1d,
            "x_coords": x_coords,
            "fwhm": fwhm,
            "abbe_limit": abbe_limit,
        }
    )

# Create figure with 2D PSFs and 1D profiles
fig = plt.figure(figsize=(16, 10))

# Top row: 2D PSFs
for i, data in enumerate(psf_data):
    ax = fig.add_subplot(2, 3, i + 1)

    # Zoom to center region
    center = len(data["x_coords"]) // 2
    crop = 80  # Pixels to show

    psf_crop = data["psf_2d"][center - crop : center + crop, center - crop : center + crop]
    extent_nm = data["x_coords"][center + crop] - data["x_coords"][center - crop]

    im = ax.imshow(
        psf_crop, cmap="hot", extent=[-extent_nm / 2, extent_nm / 2, -extent_nm / 2, extent_nm / 2]
    )
    ax.set_title(f"{data['name']}\nAbbe Limit: {data['abbe_limit']:.0f} nm", fontsize=12)
    ax.set_xlabel("Position (nm)")
    ax.set_ylabel("Position (nm)")

    # Add scale circle showing Abbe limit
    circle = plt.Circle(
        (0, 0), data["abbe_limit"] / 2, fill=False, color="white", linestyle="--", linewidth=1.5
    )
    ax.add_patch(circle)
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# Bottom: Combined 1D profiles
ax_1d = fig.add_subplot(2, 1, 2)

for data in psf_data:
    # Zoom to central region
    mask = np.abs(data["x_coords"]) < 2000  # +/-2000 nm
    ax_1d.plot(
        data["x_coords"][mask],
        data["psf_1d"][mask],
        color=data["color"],
        linewidth=2,
        label=f"{data['name']} (FWHM={data['fwhm']:.0f}nm, Abbe={data['abbe_limit']:.0f}nm)",
    )

ax_1d.axhline(y=0.5, color="gray", linestyle="--", alpha=0.5, label="Half Maximum")
ax_1d.set_xlabel("Position (nm)", fontsize=12)
ax_1d.set_ylabel("Normalized Intensity", fontsize=12)
ax_1d.set_title("PSF Cross-Sections: Higher NA = Sharper PSF = Better Resolution", fontsize=14)
ax_1d.legend(loc="upper right")
ax_1d.grid(True, alpha=0.3)
ax_1d.set_xlim(-2000, 2000)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 70)
print("PSF Summary")
print("=" * 70)
print(f"{'Objective':<20} {'NA':>8} {'Abbe (nm)':>12} {'FWHM (nm)':>12} {'Ratio':>10}")
print("-" * 70)
for data in psf_data:
    ratio = data["fwhm"] / data["abbe_limit"] if data["abbe_limit"] > 0 else 0
    print(
        f"{data['name']:<20} {data['na']:>8.2f} {data['abbe_limit']:>12.0f} {data['fwhm']:>12.0f} {ratio:>10.2f}"
    )
print("=" * 70)
print("\nNote: FWHM is smaller than Abbe limit (ratio ~0.5-0.84) because:")
print("  - FWHM measures where intensity drops to 50%")
print("  - Abbe criterion measures where two points can just be distinguished")
print("  - The exact ratio depends on sampling and calculation method")

## 3D PSF: Axial (Depth) Resolution

The PSF extends in 3D. The **axial resolution** (depth resolution) is typically worse than lateral:

$$\Delta z = \frac{2n\lambda}{\text{NA}^2}$$

This is why confocal and two-photon microscopy are used for optical sectioning.

In [ ]:
def calculate_axial_resolution(na: float, n: float, wavelength_nm: float) -> float:
    """Calculate axial resolution in nm."""
    return 2 * n * wavelength_nm / (na**2)


# Compare lateral vs axial resolution
print("Lateral vs Axial Resolution Comparison (λ = 550nm)")
print("=" * 75)
print(f"{'Objective':<20} {'NA':>6} {'n':>6} {'Lateral (nm)':>14} {'Axial (nm)':>14} {'Ratio':>10}")
print("-" * 75)

objectives_3d = [
    ("10x/0.3 Air", 0.3, 1.0),
    ("20x/0.75 Air", 0.75, 1.0),
    ("40x/0.9 Air", 0.9, 1.0),
    ("60x/1.2 Water", 1.2, 1.33),
    ("100x/1.4 Oil", 1.4, 1.515),
]

for name, na, n in objectives_3d:
    lateral = 0.61 * 550 / na
    axial = calculate_axial_resolution(na, n, 550)
    ratio = axial / lateral
    print(f"{name:<20} {na:>6.2f} {n:>6.2f} {lateral:>14.0f} {axial:>14.0f} {ratio:>10.1f}x")

print("=" * 75)
print("\nKey insight: Axial resolution is always worse than lateral (typically 2-4x).")
print("This is why 3D imaging requires special techniques (confocal, light sheet, etc.)")

---

# Section 3: Microscope Comparison with SPIDS

Now let's use SPIDS to compare actual microscope simulations using three preset objectives.

In [ ]:
# List available microscope presets
microscope_presets = list_scenario_presets("microscope")
print("Available microscope presets:")
for preset in microscope_presets:
    scenario = get_scenario_preset(preset)
    print(f"  • {preset}: {scenario.lateral_resolution_nm:.0f} nm resolution")

In [ ]:
# Load three representative presets
presets_to_compare = [
    "microscope_10x_air",  # Low magnification, low NA
    "microscope_40x_air",  # Medium magnification, high NA air
    "microscope_100x_oil",  # High magnification, oil immersion
]

scenarios = {}
microscopes = {}

print("\nMicroscope Preset Comparison")
print("=" * 90)

for preset in presets_to_compare:
    scenario = get_scenario_preset(preset)
    scenarios[preset] = scenario

    # Create instrument
    instrument_config = scenario.to_instrument_config()
    microscope = create_instrument(instrument_config)
    microscopes[preset] = microscope

# Create comparison table
print(f"\n{'Parameter':<25}", end="")
for preset in presets_to_compare:
    short_name = preset.replace("microscope_", "")
    print(f"{short_name:>20}", end="")
print()
print("-" * 85)

params = [
    ("Magnification", lambda s: f"{s._obj.magnification:.0f}x"),
    ("Numerical Aperture", lambda s: f"{s._obj.numerical_aperture:.2f}"),
    ("Immersion Medium", lambda s: s._obj.immersion_medium.capitalize()),
    ("Medium Index (n)", lambda s: f"{s._obj.medium_index:.2f}"),
    ("Wavelength", lambda s: f"{s.wavelength * 1e9:.0f} nm"),
    ("Lateral Resolution", lambda s: f"{s.lateral_resolution_nm:.0f} nm"),
    ("Axial Resolution", lambda s: f"{s.axial_resolution_um:.2f} µm"),
    ("Field of View", lambda s: f"{s.field_of_view_um:.1f} µm"),
]

for param_name, getter in params:
    print(f"{param_name:<25}", end="")
    for preset in presets_to_compare:
        value = getter(scenarios[preset])
        print(f"{value:>20}", end="")
    print()

print("=" * 85)

## Imaging the Same Target with Different Objectives

Let's image a USAF-1951 test pattern with each objective to see the resolution difference:

In [ ]:
# Create a test target
# Use a consistent field size that works for comparison
field_size = 50 * um  # 50 µm field

# We'll use the 40x preset as reference for grid size
ref_scenario = scenarios["microscope_40x_air"]
n_pixels = 1024

# Create USAF target with groups appropriate for microscopy
target = create_usaf_target(
    field_size=field_size,
    resolution=n_pixels,
    groups=(5, 6, 7),  # High-res groups
    margin_ratio=0.25,
)
ground_truth = target.generate()

print(f"Target created: {n_pixels}x{n_pixels} pixels, {field_size * 1e6:.0f} µm field")
print(f"Pixel size: {target.config.pixel_size * 1e9:.1f} nm")

In [ ]:
# Simulate imaging with each microscope
measurements = {}

for preset, microscope in microscopes.items():
    # Add batch and channel dimensions
    input_field = ground_truth.unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        measurement = microscope.forward(input_field)

    measurements[preset] = measurement.squeeze().cpu().numpy()
    print(f"Simulated {preset}")

print("\nAll simulations complete!")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
extent = [0, field_size * 1e6, 0, field_size * 1e6]  # µm

# Ground truth
axes[0, 0].imshow(ground_truth.cpu().numpy(), cmap="gray", extent=extent, origin="lower")
axes[0, 0].set_title("Ground Truth (Ideal)", fontsize=14)
axes[0, 0].set_xlabel("Position (µm)")
axes[0, 0].set_ylabel("Position (µm)")

# Microscope measurements
positions = [(0, 1), (1, 0), (1, 1)]
colors = ["blue", "green", "red"]

for (preset, meas), pos, color in zip(measurements.items(), positions, colors):
    ax = axes[pos]
    scenario = scenarios[preset]

    ax.imshow(meas, cmap="gray", extent=extent, origin="lower")

    short_name = preset.replace("microscope_", "").replace("_", " ").title()
    ax.set_title(
        f"{short_name}\nResolution: {scenario.lateral_resolution_nm:.0f} nm",
        fontsize=14,
        color=color,
    )
    ax.set_xlabel("Position (µm)")
    ax.set_ylabel("Position (µm)")

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  • 10x Air: Significant blurring, fine features not resolved")
print("  • 40x Air: Good resolution, most features visible")
print("  • 100x Oil: Excellent resolution, finest features resolved")

---

# Section 4: Nyquist Sampling Criterion

## Why Sampling Matters

Even with perfect optics, digital imaging requires **proper sampling**. The Nyquist criterion states:

$$\text{pixel size} < \frac{\lambda}{4 \cdot \text{NA}}$$

Or equivalently:

$$\text{pixel size} < \frac{\text{resolution limit}}{2.44}$$

If pixels are too large (**undersampling**), you lose information and introduce **aliasing artifacts**.

In [ ]:
def check_nyquist_sampling(wavelength_nm: float, na: float, pixel_size_nm: float) -> dict:
    """
    Check if sampling satisfies Nyquist criterion.

    Parameters
    ----------
    wavelength_nm : float
        Wavelength in nanometers
    na : float
        Numerical aperture
    pixel_size_nm : float
        Pixel size in nanometers

    Returns
    -------
    dict
        Sampling analysis results
    """
    # Nyquist limit
    nyquist_limit_nm = wavelength_nm / (4 * na)

    # Abbe resolution
    abbe_limit_nm = 0.61 * wavelength_nm / na

    # Samples per resolution element
    samples_per_res = abbe_limit_nm / pixel_size_nm

    # Status
    if pixel_size_nm <= nyquist_limit_nm:
        status = "GOOD"
        message = "Properly sampled - no aliasing"
    elif pixel_size_nm <= nyquist_limit_nm * 1.5:
        status = "MARGINAL"
        message = "Slight undersampling - minor artifacts possible"
    else:
        status = "UNDERSAMPLED"
        message = "Severe undersampling - aliasing artifacts expected"

    return {
        "nyquist_limit_nm": nyquist_limit_nm,
        "abbe_limit_nm": abbe_limit_nm,
        "pixel_size_nm": pixel_size_nm,
        "samples_per_resolution": samples_per_res,
        "status": status,
        "message": message,
    }


# Check our microscope presets
print("Nyquist Sampling Analysis for Microscope Presets")
print("=" * 85)
print(f"{'Preset':<25} {'Nyquist (nm)':>12} {'Pixel (nm)':>12} {'Samples/Res':>12} {'Status':>15}")
print("-" * 85)

for preset in presets_to_compare:
    scenario = scenarios[preset]
    microscope = microscopes[preset]

    # Get object space pixel size
    info = microscope.get_info()
    pixel_size_nm = info["object_pixel_size"] * 1e9

    result = check_nyquist_sampling(
        scenario.wavelength * 1e9, scenario._obj.numerical_aperture, pixel_size_nm
    )

    print(
        f"{preset:<25} {result['nyquist_limit_nm']:>12.1f} {pixel_size_nm:>12.1f} "
        f"{result['samples_per_resolution']:>12.1f} {result['status']:>15}"
    )

print("=" * 85)
print("\nNote: SPIDS presets are configured to satisfy Nyquist by default.")

## Visualizing Undersampling Artifacts

Let's see what happens when we undersample:

In [ ]:
# Create a high-frequency test pattern
n_full = 512
x = np.linspace(0, 10 * np.pi, n_full)
y = np.linspace(0, 10 * np.pi, n_full)
X, Y = np.meshgrid(x, y)

# High-frequency sinusoidal pattern
pattern_full = np.sin(X * 8) * np.sin(Y * 8)

# Proper sampling (Nyquist satisfied)
pattern_proper = pattern_full[::2, ::2]  # 256x256

# Undersampled (Nyquist violated)
pattern_under = pattern_full[::8, ::8]  # 64x64

# Severely undersampled
pattern_severe = pattern_full[::16, ::16]  # 32x32

# Visualize
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(pattern_full, cmap="RdBu", vmin=-1, vmax=1)
axes[0].set_title(f"Original\n{n_full}x{n_full} pixels", fontsize=12)
axes[0].axis("off")

axes[1].imshow(pattern_proper, cmap="RdBu", vmin=-1, vmax=1)
axes[1].set_title("Proper Sampling\n256x256 (2x oversample)", fontsize=12)
axes[1].axis("off")

axes[2].imshow(pattern_under, cmap="RdBu", vmin=-1, vmax=1)
axes[2].set_title("Undersampled\n64x64 (aliasing visible)", fontsize=12)
axes[2].axis("off")

axes[3].imshow(pattern_severe, cmap="RdBu", vmin=-1, vmax=1)
axes[3].set_title("Severe Undersampling\n32x32 (pattern destroyed)", fontsize=12)
axes[3].axis("off")

plt.suptitle(
    "Nyquist Sampling: Effect of Pixel Size on High-Frequency Patterns", fontsize=14, y=1.02
)
plt.tight_layout()
plt.show()

print("\nKey observations:")
print("  • Proper sampling preserves the original pattern")
print("  • Undersampling creates false low-frequency patterns (aliasing)")
print("  • Severe undersampling completely destroys the original information")
print("  \n→ Always ensure your camera pixels are smaller than Nyquist limit!")

---

# Assessment Questions

Test your understanding with these questions:

In [ ]:
print("=" * 70)
print("ASSESSMENT QUESTIONS")
print("=" * 70)

print("\n📝 Question 1:")
print("   Calculate the resolution for a 60x/1.2 NA water immersion")
print("   objective at 550nm wavelength.")
print("   ")
print("   Your calculation:")
q1_result = calculate_resolution(550, 1.2)
print(f"   Δx = 0.61 × 550 / 1.2 = {q1_result['resolution_nm']:.0f} nm")
print("   ✅ Expected: ~280 nm")

print("\n" + "-" * 70)

print("\n📝 Question 2:")
print("   Which objective would you need to resolve features of 400nm?")
print("   ")
print("   Analysis:")
print("   For resolution = 400nm at λ=550nm:")
print("   NA = 0.61 × 550 / 400 = 0.84")
print("   ✅ Need NA > 0.84 → Use 40x/0.9 Air or higher NA objective")

print("\n" + "-" * 70)

print("\n📝 Question 3:")
print("   Why is oil immersion required for NA > 1.0?")
print("   ")
print("   Answer:")
print("   NA = n × sin(θ)")
print("   • For air (n=1.0): max NA = 1.0 × sin(90°) = 1.0")
print("   • For oil (n=1.515): max NA = 1.515 × sin(90°) = 1.515")
print("   ")
print("   ✅ Oil's higher refractive index allows collection of light at")
print("   steeper angles, enabling NA > 1.0 for better resolution.")

print("\n" + "=" * 70)

---

# Summary

## Key Takeaways

### 1. The Abbe Diffraction Limit
$$\Delta x = \frac{0.61 \lambda}{\text{NA}}$$
- Resolution is **fundamentally limited** by diffraction
- No amount of magnification can exceed this limit

### 2. Numerical Aperture (NA) is Key
- Higher NA = Better resolution
- NA = n * sin(theta), where n is medium refractive index
- Oil immersion enables NA > 1.0

### 3. Point Spread Function (PSF)
- PSF describes how a point is imaged
- PSF FWHM is related to but smaller than the Abbe limit
- Axial resolution is typically 2-4x worse than lateral

### 4. Nyquist Sampling
- Pixel size must be < lambda/(4*NA) to avoid aliasing
- Undersampling destroys high-frequency information
- SPIDS presets are configured for proper sampling

## Objective Selection Guide

| Resolution Needed | Objective Type | Use Case |
|-------------------|----------------|----------|
| > 1 um | 10x/0.3 Air | Sample overview, tissue sections |
| 400-1000 nm | 20x-40x Air | Cells, bacteria, large structures |
| 250-400 nm | 40x-60x Air/Water | Cellular details, organelles |
| < 250 nm | 100x Oil | Subcellular structures, viruses |

---

## Reference Documentation

For detailed parameter lookup and formulas:

- **[Optical Resolution Limits](../../docs/references/optical_resolution_limits.md)** - Abbe, Rayleigh, Sparrow criteria
- **[Microscopy Parameters](../../docs/references/microscopy_parameters.md)** - Objective specs, immersion media
- **[Scenario Preset Catalog](../../docs/references/scenario_preset_catalog.md)** - All 9 microscope presets

---

## Next Steps

- **[Learning 2: Resolution Validation](learning_02_resolution_validation.ipynb)**: Validate resolution using USAF-1951 targets
- **[Learning 3: Illumination Modes](learning_03_illumination_modes.ipynb)**: Explore brightfield, phase contrast, DIC
- **[Quickstart 1: Microscopy Basic](quickstart_01_microscopy_basic.ipynb)**: Hands-on microscope simulation

In [ ]:
print("\n" + "=" * 70)
print("🎉 Congratulations! You've completed Learning 1: Resolution Fundamentals")
print("=" * 70)
print("\nYou now understand:")
print("  ✅ The Abbe diffraction limit and how to calculate it")
print("  ✅ How NA and wavelength affect resolution")
print("  ✅ The role of immersion media (air, water, oil)")
print("  ✅ Point Spread Functions and their significance")
print("  ✅ Nyquist sampling requirements for digital imaging")
print("\nReady for Learning 2? Learn to validate resolution with USAF-1951 targets!")